# Tree segmentation with multitemporal Sentinel 1/2 imagery

## John Brandt
## April 02, 2020

## Overview

This notebook contains the TensorFlow model training and prediction used to segment trees for [Restoration Mapper](https://restorationmapper.org). The notebook uses tensorflow 1.13.1 and additionally relies on Keras and tflearn. 

## Table of Contents

- Package loading
- Utility scripts
- Hyperparameter definitions
- Custom tensorflow layer functions
- Tensorflow graph creation
- Data loading
- Data preprocessing
- Equibatch creation
- Loss definition
- Tensorflow graph initialization
- Training
- Model validation
- Sanity Checks

## Package Loading

In [ ]:
from tqdm import tqdm_notebook, tnrange
import tensorflow as tf
#import tensorflow_probability as tfp


sess = tf.Session()
from keras import backend as K
K.set_session(sess)

from time import sleep

import keras
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.layers import ELU
from keras.losses import binary_crossentropy
from tensorflow.python.ops import array_ops
from tensorflow.python.keras.layers import Conv2D, Lambda, Dense, Multiply, Add
from tensorflow.initializers import glorot_normal, lecun_normal
from scipy.ndimage import median_filter
from skimage.transform import resize

import pandas as pd
import numpy as np
from random import shuffle
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random
import itertools
from tensorflow.contrib.framework import arg_scope
from keras.regularizers import l1
from tensorflow.layers import batch_normalization
from tensorflow.python.util import deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

os.environ['KMP_DUPLICATE_LIB_OK']='True'

## Utility scripts

In [ ]:
%run ../src/layers/zoneout.py
%run ../src/layers/convgru.py
%run ../src/layers/adabound.py
%run ../src/layers/dropblock.py
%run ../src/layers/extra_layers.py
%run ../src/preprocessing/indices.py
%run ../src/preprocessing/slope.py
%run ../src/utils/metrics.py
%run ../src/not_for_release/lovasz.py

# Hyperparameter definitions

In [ ]:
ZONE_OUT_PROB = 0.20 #0.2 for master model
GRU_FILTERS = 32
FPA_FILTERS = 28
OUT_FILTERS = 32
AVERAGE_GRU_FILTERS = True
ACTIVATION_FUNCTION = 'relu'

INITIAL_LR = 1e-4
FINAL_LR = 2e-2
DROPBLOCK_MAXSIZE = 5
DECONV = 'bilinear'
N_CONV_BLOCKS = 1
FINAL_ALPHA = 0.7
LABEL_SMOOTHING = 0.10 #0.075 for master
BATCH_RENORM = 'norm'

L2_REG = 0.0
BN_MOMENTUM = 0.9
BATCH_SIZE = 16
MAX_DROPBLOCK = 0.85

gru_flt = 32
fpa_flt = 28
out_conv_flt = 32

IMAGE_SIZE = 16
LABEL_SIZE = 14

# Hyperparameter grid search definitions

In [ ]:
# There are 3.8 million combinations of hyperparameters
params = {
    'zone_out' : [0],#range(0, 0.5, 0.1), #6
    'average_gru_filters': [True, False], # 2
    'final_lr': [5e-3, 1e-2, 2e-2, 5e-2], # 4
    'n_conv_blocks': [3, 2, 1], # 3
    'final_alpha': [0.8, 0.75, 0.7, 0.65, 0.6], #5
    'label_smoothing': [0., 0.025, 0.05, 0.075, 0.10, 0.125],
    'batch_renorm': ['renorm', 'norm']
}

# Custom layer definitions

### Utility blocks (Batch norm, cSSE, etc.)

In [ ]:
def Batch_Normalization(x, training, scope, clipping_params):
    '''Batch renormalization layer from tensorflow.layers

         Parameters:
          x (tf.Variable): input layer
          training (tf.Variable): flag to differentiate between train/test ops
          scope (str): tensorflow scope
          clipping_params (dict): specifies clipping of 
                                  rmax, dmax, rmin for renormalization

         Returns:
          x (tf.Variable): output of batch renormalization
          
         References:
          https://github.com/tensorflow/docs/blob/r1.13/site/en/
          api_docs/python/tf/layers/batch_normalization.md
    '''
    return batch_normalization(inputs=x, 
                               momentum = BN_MOMENTUM, 
                               training=training,
                               renorm = True,
                               reuse=None,
                               renorm_clipping = clipping_params,
                               name = scope)


def calc_renorm_params(epoch, n_samples, batch_size, k = 0):
    '''Calculates the clipping parameters for renormalization
       based on the learning schedule outlined in the original paper
       where rmax is initialized to 1, dmax to 0, rmin to 0,
       (effectively batch normalization), and then rmax and dmax
       are gradually reaxed to 3 and 5 over 40k and 25k steps,
       respectively. In this case, 40k has been reduced to 30k, and 25k
       has been reduced to 20k.

         Parameters:
          epoch (int): number of current training epoch
                       if testing, epoch number of model used
          n_samples (int): total number of training samples
          batch_size (int): training batch size 

         Returns:
          rmax (float)
          dmax (float)
          rmin (float)
    '''
    step = epoch * (n_samples // batch_size)
    step += k
    if step < 2500:
        rmax = 1.
        dmax = 0.
        rmin = 0.
    if step >= 2500:
        rmax = np.min([1 + 2*((step-2500)/20000), 3])
        dmax = np.min([1 + 5*((step-2500)/15000), 5])
        rmin = 0.
    return rmax, dmax, rmin

def cse_block(prevlayer, prefix):
    '''Channel excitation and spatial squeeze layer. 
       Calculates the mean of the spatial dimensions and then learns
       two dense layers, one with relu, and one with sigmoid, to rerank the
       input channels
       
         Parameters:
          prevlayer (tf.Variable): input layer
          prefix (str): prefix for tensorflow scope

         Returns:
          x (tf.Variable): output of the cse_block
    '''
    mean = Lambda(lambda xin: K.mean(xin, axis=[1, 2]))(prevlayer)
    lin1 = Dense(K.int_shape(prevlayer)[3] // 2, name=prefix + 'cse_lin1', activation='relu')(mean)
    lin2 = Dense(K.int_shape(prevlayer)[3], name=prefix + 'cse_lin2', activation='sigmoid')(lin1)
    x = Multiply()([prevlayer, lin2])
    return x


def sse_block(prevlayer, prefix):
    '''Spatial excitation and channel squeeze layer.
       Calculates a 1x1 convolution with sigmoid activation to create a 
       spatial map that is multiplied by the input layer

         Parameters:
          prevlayer (tf.Variable): input layer
          prefix (str): prefix for tensorflow scope

         Returns:
          x (tf.Variable): output of the sse_block
    '''
    conv = Conv2D(1, (1, 1), padding="same", kernel_initializer="glorot_uniform",
                  activation='sigmoid', strides=(1, 1),
                  name=prefix + "_conv")(prevlayer)
    conv = Multiply(name=prefix + "_mul")([prevlayer, conv])
    return conv


def csse_block(x, prefix):
    '''Implementation of Concurrent Spatial and Channel 
       ‘Squeeze & Excitation’ in Fully Convolutional Networks
    
        Parameters:
          prevlayer (tf.Variable): input layer
          prefix (str): prefix for tensorflow scope

         Returns:
          x (tf.Variable): added output of cse and sse block
          
         References:
          https://arxiv.org/abs/1803.02579
    '''
    cse = cse_block(x, prefix)
    sse = sse_block(x, prefix)
    x = Add(name=prefix + "_csse_mul")([cse, sse])

    return x

class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

### Conv GRU Block

In [ ]:
def gru_block(inp, length, size, flt, scope, train, normalize = True):
    '''Bidirectional convolutional GRU block with 
       zoneout and CSSE blocks in each time step

         Parameters:
          inp (tf.Variable): (B, T, H, W, C) layer
          length (tf.Variable): (B, T) layer denoting number of
                                steps per sample
          size (int): kernel size of convolution
          flt (int): number of convolution filters
          scope (str): tensorflow variable scope
          train (tf.Bool): flag to differentiate between train/test ops
          normalize (bool): whether to compute layer normalization

         Returns:
          gru (tf.Variable): (B, H, W, flt*2) bi-gru output
          steps (tf.Variable): (B, T, H, W, flt*2) output of each step
    '''
    with tf.variable_scope(scope):
        print(f"GRU input shape {inp.shape}, zoneout: {ZONE_OUT_PROB}")
        cell_fw = ConvGRUCell(shape = size, filters = flt,
                           kernel = [3, 3], padding = 'VALID', normalize = normalize, sse = True)
        cell_bw = ConvGRUCell(shape = size, filters = flt,
                           kernel = [3, 3], padding = 'VALID', normalize = normalize, sse = True)
        cell_fw = ZoneoutWrapper(
           cell_fw, zoneout_drop_prob = ZONE_OUT_PROB, is_training = train)
        cell_bw = ZoneoutWrapper(
            cell_bw, zoneout_drop_prob = ZONE_OUT_PROB, is_training = train)
        steps, out = convGRU(inp, cell_fw, cell_bw, length)
        gru = tf.concat(out, axis = -1)
        steps = tf.concat(steps, axis = -1)
        print(f"Down block output shape {gru.shape}")
    return gru, steps

### Conv blocks

In [ ]:
def conv_selu(inp, is_training, kernel_size, scope,
                filter_count = 16, pad = True, padding = 'valid', dilated = False,
                activation = True):
    '''Convolutional 2D layer with SELU activation and Lecun normal initialization
       with no batch norm. Only used if params['activation'] = 'selu'

         Parameters:
          inp (tf.Variable): (B, H, W, C) input layer
          is_training (str): flag to differentiate between train/test ops
          kernel_size (int): kernel size of convolution
          scope (str): tensorflow variable scope
          filter_count (int): number of convolution filters
          pad (bool): whether or not to reflect pad input
          padding (str): one of ['valid', 'same']
          dilated (bool): whether to perform atruous convolution
          activation (bool): whether to activate output

         Returns:
          conv (tf.Variable): output of Conv2D -> SELU
          
         References:
          https://arxiv.org/abs/1706.02515
    '''
    if activation:
        act = selu
    else:
        act = None
    if not dilated:
        padded = ReflectionPadding2D((1, 1,))(inp)
        conv = Conv2D(filters = filter_count, kernel_size = (kernel_size, kernel_size), activation = act,
                        padding = padding, kernel_initializer = lecun_normal())(padded)
    if not dilated and not pad:
        conv = Conv2D(filters = filter_count, kernel_size = (kernel_size, kernel_size), activation = act,
                        padding = padding, kernel_initializer = lecun_normal())(inp)
    if dilated:
        padded = ReflectionPadding2D((2, 2,))(inp)
        conv = Conv2D(filters = filter_count, kernel_size = (3, 3), activation = act, dilation_rate = (2, 2),
                        padding = padding, kernel_initializer = lecun_normal())(padded)
    return conv

def conv_bn_relu(inp, 
                 is_training, 
                 kernel_size,
                 scope,
                 filters, 
                 clipping_params,
                 keep_rate,
                 stride = (1, 1),
                 activation = True,
                 use_bias = False,
                 batch_norm = True,
                 dropblock = True,
                 csse = True):
    '''2D convolution, batch renorm, relu block, 3x3 drop block. 
       Use_bias must be set to False for batch normalization to work. 
       He normal initialization is used with batch normalization.
       RELU is better applied after the batch norm.
       DropBlock performs best when applied last, according to original paper.

         Parameters:
          inp (tf.Variable): input layer
          is_training (str): flag to differentiate between train/test ops
          kernel_size (int): size of convolution
          scope (str): tensorflow variable scope
          filters (int): number of filters for convolution
          clipping_params (dict): specifies clipping of 
                                  rmax, dmax, rmin for renormalization
          activation (bool): whether to apply RELU
          use_bias (str): whether to use bias. Should always be false

         Returns:
          bn (tf.Variable): output of Conv2D -> Batch Norm -> RELU
        
         References:
          http://papers.nips.cc/paper/8271-dropblock-a-regularization-
              method-for-convolutional-networks.pdf
          https://arxiv.org/abs/1702.03275
          
    '''
    
    bn_flag = "Batch Renorm" if batch_norm else ""
    activation_flag = "RELU" if activation else "Linear"
    csse_flag = "CSSE" if csse else "No CSSE"
    bias_flag = "Bias" if use_bias else "NoBias"
    drop_flag = "DropBlock" if dropblock else "NoDrop"
        
    
    print("{} {} Conv 2D {} {} {} {} {}".format(scope, kernel_size,
                                                   bn_flag, activation_flag,
                                                   csse_flag, bias_flag, drop_flag))
    
    with tf.variable_scope(scope + "_conv"):
        conv = Conv2D(filters = filters, kernel_size = (kernel_size, kernel_size),  strides = stride,
                      activation = None, padding = 'valid', use_bias = use_bias,
                      kernel_initializer = tf.keras.initializers.he_normal())(inp)
    if batch_norm:
        conv = Batch_Normalization(conv, is_training, scope, clipping_params)
    if activation:
        conv = tf.nn.relu(conv)
    if csse:
        conv = csse_block(conv, 'csse_' + scope)
    if dropblock: 
        with tf.variable_scope(scope + "_drop"):
            drop_block = DropBlock2D(keep_prob=keep_rate, block_size=4)
            conv = drop_block(conv, is_training)
    return conv


def create_deconv_init(filter_size, num_channels):
    '''Initializes a kernel weight matrix with a bilinear deconvolution
    
         Parameters:
          filter_size (int): kernel size of convolution
          num_channels (int): number of filters for convolution

         Returns:
          bilinear_init (tf.Variable): [filter_size, filter_size, num_channels] kernel
    '''
    bilinear_kernel = np.zeros([filter_size, filter_size], dtype=np.float32)
    scale_factor = (filter_size + 1) // 2
    if filter_size % 2 == 1:
        center = scale_factor - 1
    else:
        center = scale_factor - 0.5
    for x in range(filter_size):
        for y in range(filter_size):
            bilinear_kernel[x,y] = (1 - abs(x - center) / scale_factor) * \
                                   (1 - abs(y - center) / scale_factor)
    weights = np.zeros((filter_size, filter_size, num_channels, num_channels))
    for i in range(num_channels):
        weights[:, :, i, i] = bilinear_kernel

    #assign numpy array to constant_initalizer and pass to get_variable
    bilinear_init = tf.constant_initializer(value=weights, dtype=tf.float32)
    return bilinear_init

def get_deconv2d(inp, filter_count, num_channels, scope, is_training, clipping_params):
    '''Creates a deconvolution layer with Conv2DTranspose. Following recent
       recommendations to use 4 kernel, 2 stride to avoid artifacts. 
       Initialize kernel with bilinear upsampling.

         Parameters:
          inp (tf.Variable): input tensorflow layer (B, X, Y, C) shape
          filter_count (int): number of filters for convolution
          num_channels (int): number of output channels
          scope (str): tensorflow variable scope
          is_training (str): flag to differentiate between train/test ops
          clipping_params (dict): specifies clipping of 
                                  rmax, dmax, rmin for renormalization

         Returns:
          x (tf.Variable): layer with (B, x * 2, y * 2, C) shape
          
         References:
          https://distill.pub/2016/deconv-checkerboard/
    '''
    bilinear_init = create_deconv_init(4, filter_count)
    x = tf.keras.layers.Conv2DTranspose(filters = filter_count, kernel_size = (4, 4),
                                        strides=(2, 2), padding='same', 
                                        use_bias = False,
                                        kernel_initializer = bilinear_init)(inp)
    #x = ELU()(x)
    #x = tf.nn.relu(x)
    x = Batch_Normalization(x, training=is_training, scope = scope + "bn", clipping_params = clipping_params)
    return x

In [ ]:
def fpa(inp, is_training, filter_count, clipping_params, 
        keep_rate, upsample = "upconv"):
    '''Feature pyramid attention layer block, that allows for cross-scale combination
       of different size features without making blurry feature maps.

         Parameters:
          inp (tf.Variable): input tensorflow layer
          is_training (str): flag to differentiate between train/test ops
          filter_count (int): number of filters for convolution
          clipping_params (dict): specifies clipping of 
                                  rmax, dmax, rmin for renormalization

         Returns:
          concat_1 (tf.Variable): output of FPA
          
         References:
          https://arxiv.org/abs/1805.10180
    '''
    one = conv_bn_relu(inp = inp, is_training = is_training, 
                       kernel_size = 1, scope =  'forward1',
                       filters = filter_count, clipping_params = clipping_params,
                       keep_rate = keep_rate, activation = False,
                       use_bias = False, batch_norm = True,
                       dropblock = False,
                       csse = False)
    inp_pad = ReflectionPadding2D(padding = (2, 2))(inp)
    seven = conv_bn_relu(inp = inp_pad, is_training = is_training, 
                       kernel_size = 5, scope =  'down1', stride = (2, 2),
                       filters = filter_count, clipping_params = clipping_params,
                       keep_rate = keep_rate, activation = True,
                       use_bias = False, batch_norm = True, csse = True, dropblock = False)
    seven_pad = ReflectionPadding2D(padding = (2, 2))(seven)
    seven_f = conv_bn_relu(inp = seven_pad, is_training = is_training, 
                       kernel_size = 5, scope =  'down1_f',
                       filters = filter_count, clipping_params = clipping_params,
                       keep_rate = keep_rate, activation = False,
                       use_bias = False, batch_norm = True, csse = False, dropblock = False)
    
    print("Seven: {}".format(seven.shape))
    print("Seven f: {}".format(seven_f.shape))
    
    five_pad = ReflectionPadding2D(padding = (1, 1))(seven)
    five = conv_bn_relu(inp = five_pad, is_training = is_training,  stride = (2, 2),
                       kernel_size = 3, scope =  'down2',
                       filters = filter_count, clipping_params = clipping_params,
                       keep_rate = keep_rate, activation = True,
                       use_bias = False, batch_norm = True, csse = True, dropblock = False)
    
    five_pad2 = ReflectionPadding2D(padding = (1, 1))(five)
    five_f = conv_bn_relu(inp = five_pad2, is_training = is_training, 
                       kernel_size = 3, scope =  'down2_f',
                       filters = filter_count, clipping_params = clipping_params,
                       keep_rate = keep_rate, activation = False,
                       use_bias = False, batch_norm = True, csse = False, dropblock = False)
    print("Five: {}".format(five.shape))
    print("Five_F: {}".format(five_f.shape))
    '''
    three_pad = ReflectionPadding2D(padding = (1, 1))(five)
    three = conv_bn_relu(inp = three_pad, is_training = is_training,  stride = (2, 2),
                       kernel_size = 3, scope =  'down3',
                       filters = filter_count, clipping_params = clipping_params,
                       keep_rate = keep_rate, activation = True,
                       use_bias = False, batch_norm = True, csse = True, dropblock = False)
    
    three_pad2 = ReflectionPadding2D(padding = (1, 1))(three)
    three_f = conv_bn_relu(inp = three_pad2, is_training = is_training, 
                       kernel_size = 3, scope =  'down3_f',
                       filters = filter_count, clipping_params = clipping_params,
                       keep_rate = keep_rate, activation = False,
                       use_bias = False, batch_norm = True, csse = True, dropblock = False)
        
    
    if upsample == 'upconv' or 'bilinear':
        three_up = tf.keras.layers.UpSampling2D((2, 2), interpolation = 'bilinear')(three_f)
        if upsample == 'upconv':
            three_up = ReflectionPadding2D((1, 1,))(three_up)
            three_up = conv_bn_relu(inp = three_up, is_training = is_training, 
                       kernel_size = 3, scope =  'upconv1',
                       filters = filter_count, clipping_params = clipping_params,
                       keep_rate = keep_rate, activation = True,
                       use_bias = False, batch_norm = True,
                       csse = False, dropblock = False)
            
            # 4x4
            three_up = tf.nn.relu(tf.add(three_up, five_f))
    '''        
    
    if upsample == 'upconv' or "bilinear":
        five_up = tf.keras.layers.UpSampling2D((2, 2), interpolation = 'nearest')(five)
        if upsample == 'upconv':
            five_up = ReflectionPadding2D((1, 1,))(five_up)
            five_up = conv_bn_relu(inp = five_up, is_training = is_training, 
                       kernel_size = 3, scope =  'upconv2',
                       filters = filter_count, clipping_params = clipping_params,
                       keep_rate = keep_rate, activation = True,
                       use_bias = False, batch_norm = True, 
                       csse = False, dropblock = False)
            five_up = tf.nn.relu(tf.add(five_up, seven_f))
            
    if upsample == 'upconv' or "bilinear":
        seven_up = tf.keras.layers.UpSampling2D((2, 2), interpolation = 'nearest')(five_up)
        if upsample == 'upconv':
            seven_up = ReflectionPadding2D((1, 1,))(seven_up)
            seven_up = conv_bn_relu(inp = seven_up, is_training = is_training, 
                       kernel_size = 3, scope =  'upconv3',
                       filters = filter_count, clipping_params = clipping_params,
                       keep_rate = keep_rate, activation = True,
                       use_bias = False, batch_norm = True, 
                       csse = False, dropblock = False)
    
    print("One: {}".format(one.shape))
    print("Five_up: {}".format(five_up.shape))
    print("Seven_up: {}".format(seven_up.shape))
    
    # top block

    #pooled = tf.keras.layers.GlobalAveragePooling2D()(inp)
    #one_top = conv_bn_relu(inp = tf.reshape(pooled, (-1, 1, 1, pooled.shape[-1])),
    #                       is_training = is_training, 
    #                   kernel_size = 1, scope =  'topconv',
    #                   filters = filter_count, clipping_params = clipping_params,
    #                   keep_rate = keep_rate, activation = False,
    #                   use_bias = False, batch_norm = True, 
    #                   csse = False, dropblock = False)
    #one_top = conv_bn_relu(tf.reshape(pooled, (-1, 1, 1, pooled.shape[-1])),
    ##                      is_training, 1, 'top1', filter_count, pad = False)
    #four_top = tf.keras.layers.UpSampling2D((16, 16))(one_top)
    
    #seven_up = tf.multiply(one, seven_up)
    out = tf.nn.relu(tf.multiply(seven_up, one))
    return out



# Model definition

## Placeholders

In [ ]:
n_bands = 16 # 16 for master model
reg = keras.regularizers.l2(L2_REG) # for GRU
inp = tf.placeholder(tf.float32, shape=(None, 12, IMAGE_SIZE, IMAGE_SIZE, n_bands))
length = tf.placeholder(tf.int32, shape = (None, 1))
labels = tf.placeholder(tf.float32, shape=(None, 14, 14))#, 1))
keep_rate = tf.placeholder_with_default(1.0, ()) # For DropBlock
length2 = tf.reshape(length, (-1,)) # Remove
is_training = tf.placeholder_with_default(False, (), 'is_training') # For BN, DropBlock
alpha = tf.placeholder(tf.float32, shape = ()) # For loss scheduling
ft_lr = tf.placeholder_with_default(0.001, shape = ()) # For loss scheduling
loss_weight = tf.placeholder_with_default(1.0, shape = ())
beta_ = tf.placeholder_with_default(0.0, shape = ())


inp_median = tf.placeholder(tf.float32, shape = (None, IMAGE_SIZE, IMAGE_SIZE, n_bands))

In [ ]:
rmax = tf.placeholder(tf.float32, shape = ())
rmin = tf.placeholder(tf.float32, shape = ())
dmax = tf.placeholder(tf.float32, shape = ())

clipping_params = {
    'rmax': rmax,
    'rmin': rmin,
    'dmax': dmax
}

## Layers

In [ ]:
####### Encoder ###########
# GRU       24 x 24 x 24 x 2
# Conv      24 x 24 x 48

# Pool      12 x 12 
# Conv      12 x 12 x 64

# Pool      6 x 6   
# Conv      4 x 4 x 96



####### Decoder ###########
# Upsample  8 x 8 
# Concat    8 x 8 (12 x 12)
# Conv      8 x 8

# Upsample 16 x 16
# Concat   16 x 16 (24 x 24)
# Conv     14 x 14

gru, _ = gru_block(inp = inp, length = length2,
                            size = [24, 24],
                            flt = 28,
                            scope = 'down_16',
                            train = is_training)

gru = ReflectionPadding2D((1, 1,))(gru)
gru_conv = conv_bn_relu(inp = gru, is_training = is_training, stride = (1, 1),
            kernel_size = 3, scope = 'mean', filters = 48, clipping_params = clipping_params,
            keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
            csse = True, dropblock = True)

pool1 = MaxPool2D()(gru_conv)
conv1 = conv_bn_relu(inp = pool1, is_training = is_training, stride = (1, 1),
            kernel_size = 3, scope = 'conv1', filters = 64, clipping_params = clipping_params,
            keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
            csse = True, dropblock = True)

pool2 = MaxPool2D()(conv1)
conv2 = conv_bn_relu(inp = pool2, is_training = is_training, stride = (1, 1),
            kernel_size = 3, scope = 'conv2', filters = 96, clipping_params = clipping_params,
            keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
            csse = True, dropblock = True)


# Decoder 4 - 8
up1 = tf.keras.layers.UpSampling2D((2, 2), interpolation = 'nearest')(conv2)
up1 = ReflectionPadding2D((1, 1,))(up1)
up1 = conv_bn_relu(inp = up1, is_training = is_training, stride = (1, 1),
                    kernel_size = 3, scope = 'up1', filters = 64, clipping_params = clipping_params,
                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
                    csse = True, dropblock = True)
conv1crop = Crop2D(2)(conv1)
up1 = tf.concat([up1, conv1crop], axis = -1)
up1 = ReflectionPadding2D((1, 1,))(up1)
up1 = conv_bn_relu(inp = up1, is_training = is_training, stride = (1, 1),
                    kernel_size = 3, scope = 'up1conv', filters = 64, clipping_params = clipping_params,
                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
                    csse = True, dropblock = True)

# Decoder 8 - 16
up2 = tf.keras.layers.UpSampling2D((2, 2), interpolation = 'nearest')(up1)
up2 = ReflectionPadding2D((1, 1,))(up2)
up2 = conv_bn_relu(inp = up2, is_training = is_training, stride = (1, 1),
                    kernel_size = 3, scope = 'up2', filters = 48, clipping_params = clipping_params,
                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
                    csse = True, dropblock = True)
conv2crop = Crop2D(4)(gru_conv)
up2 = tf.concat([up2, conv2crop], axis = -1)
up2 = conv_bn_relu(inp = up2, is_training = is_training, stride = (1, 1),
                    kernel_size = 3, scope = 'up2conv', filters = 48, clipping_params = clipping_params,
                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
                    csse = True, dropblock = True)





In [ ]:
gru_out, steps = gru_block(inp = inp, length = length2,
                            size = [16, 16],
                            flt = gru_flt,
                            scope = 'down_16',
                            train = is_training)

median = ReflectionPadding2D((1, 1,))(inp_median)
median = conv_bn_relu(inp = median, is_training = is_training, stride = (1, 1),
                    kernel_size = 3, scope = 'mean', filters = 32, clipping_params = clipping_params,
                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
                    csse = True, dropblock = True)
print("median1", median.shape)

median = tf.concat([gru_out, median], axis = -1)
print("median", median.shape)
median = conv_bn_relu(inp = median, is_training = is_training, stride = (1, 1),
                    kernel_size = 1, scope = 'mean2', filters = 48, clipping_params = clipping_params,
                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
                    csse = False, dropblock = False)

print("median", median.shape)

#mean_2f = conv_bn_relu(inp = mean_1f, is_training = is_training, stride = (1, 1),
#                    kernel_size = 1, scope = 'mean_1f', filters = 64, clipping_params = clipping_params,
#                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
#                    csse = False, dropblock = False)

max1 = MaxPool2D()(median)
print("max1", max1.shape)


# Down block 2 (8 - 4)
down2 = conv_bn_relu(inp = max1, is_training = is_training, stride = (1, 1),
                    kernel_size = 3, scope = 'down2', filters = 64, clipping_params = clipping_params,
                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
                    csse = True, dropblock = True)
print("down2", down2.shape)
down2_f = conv_bn_relu(inp = down2, is_training = is_training, stride = (1, 1),
                    kernel_size = 3, scope = 'down2_f', filters = 64, clipping_params = clipping_params,
                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
                    csse = True, dropblock = True)
print("Down2_f", down2_f.shape)

# Upblock 1 (4 - 8)
up1 = tf.keras.layers.UpSampling2D((2, 2), interpolation = 'nearest')(down2_f)
up1 = ReflectionPadding2D((1, 1,))(up1)
up1 = conv_bn_relu(inp = up1, is_training = is_training, stride = (1, 1),
                    kernel_size = 3, scope = 'up1', filters = 48, clipping_params = clipping_params,
                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
                    csse = True, dropblock = True)
up1 = tf.concat([up1, max1], axis = -1)
print("up1", up1.shape)


# Upblock2 (8 - 16)
up2 = tf.keras.layers.UpSampling2D((2, 2), interpolation = 'nearest')(up1)
up2 = ReflectionPadding2D((1, 1,))(up2)
up2 = conv_bn_relu(inp = up2, is_training = is_training, stride = (1, 1),
                    kernel_size = 3, scope = 'up2', filters = 48, clipping_params = clipping_params,
                    keep_rate = keep_rate, activation = True, use_bias = False, batch_norm = True,
                    csse = True, dropblock = True)
up2 = tf.concat([up2, median], axis = -1)
print("up2", up2.shape)


x = conv_bn_relu(inp = up2,
                 is_training = is_training,
                 kernel_size = 3,
                 scope = "outconv2",
                 filters = 48,
                 clipping_params = clipping_params,
                 activation = True,
                 keep_rate = keep_rate,
                 use_bias = False, dropblock = False,
                 batch_norm = True, csse = False)
print("Finzl output shape: ", x.shape)

print("Initializing last sigmoid bias with -2.94 constant")
init = tf.constant_initializer([-np.log(0.7/0.3)]) # For focal loss
fm = Conv2D(filters = 1,
            kernel_size = (1, 1),
            padding = 'valid',
            activation = 'sigmoid',
            bias_initializer = init,
           )(x) # For focal loss

In [ ]:
# 24, 12, 6, 12, 24

In [ ]:
total_parameters = 0
for variable in tf.trainable_variables():
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    total_parameters += variable_parameters
print(f"This model has {total_parameters} parameters")

# Data loading

*  Load in CSV data from Collect Earth
*  Reconstruct the X, Y grid for the Y data per sample
*  Calculate NDVI, EVI, SAVI, BI, MSAVI2, and SI
*  Stack X, Y, length data
*  Apply median filter to DEM

In [ ]:
import hickle as hkl
normalize = False
train_x = hkl.load("../tile_data/processed/train_x.hkl")
train_y = hkl.load("../tile_data/processed/train_y.hkl")
train_l = hkl.load("../tile_data/processed/train_l.hkl")

train_x = np.delete(train_x, 14, -1) # test not deleting the SI inde

data = pd.read_csv("train_plot_ids.csv")

In [ ]:
ethiopia_x =  hkl.load("../tile_data/processed/ethiopia/train_x.hkl")
ethiopia_y =  hkl.load("../tile_data/processed/ethiopia/train_y.hkl")
ethiopia_l =  hkl.load("../tile_data/processed/ethiopia/train_l.hkl")
ethiopia_x = np.delete(ethiopia_x, 14, -1)

train_x = np.concatenate([train_x, ethiopia_x])
train_y = np.concatenate([train_y, ethiopia_y])
train_l = np.concatenate([train_l, ethiopia_l])


# Data preprocessing

*  Identify and remove samples with time steps / channels that have a 0. or 1. value, which indicates missing data
*  Identify and remove samples with time steps / channels with no variation, which indicates missing data
*  Identify and remove samples with values above or below the allowable values for the band
*  Identify and remove samples with null data, or samples with extreme band 0 data (which squash all the "clean" samples)
*  Smooth per-pixel temporal data with Whittaker smoother, d = 2, lambda = 0.5 to reduce sample noise

In [ ]:
below_1 = [i for i, val in enumerate(train_x[..., :10]) if np.min(val) < -2]
above_1 = [i for i, val in enumerate(train_x[..., :10]) if np.max(val) > 2]
min_vals = [np.min(val) for i, val in enumerate(train_x[..., :10]) if np.min(val) < -1.5]
max_vals = [np.max(val) for i, val in enumerate(train_x[..., :10]) if np.max(val) > 1.5]
nans = [i for i, val in enumerate(train_x) if np.sum(np.isnan(val)) > 100]
oob_vals = [i for i, val in enumerate(train_x) if np.max(val[..., 0]) > 0.7]

outliers = below_1 + above_1 + nans + oob_vals
outliers = list(set(outliers))
print("Removing {} outlying training data points".format(len(outliers)))
print(sorted(outliers))
train_x = np.delete(train_x, outliers, 0)
train_y = np.delete(train_y, outliers, 0)
train_l = np.delete(train_l, outliers)
data = data.drop(outliers, 0)
data.reset_index(inplace = True, drop = True)

outliers = [4767, 4769, 4771, 4772, 4773, 4783, 4788, 4789]
print("Removing {} outlying training data points".format(len(outliers)))
print(sorted(outliers))
train_x = np.delete(train_x, outliers, 0)
train_y = np.delete(train_y, outliers, 0)
train_l = np.delete(train_l, outliers)

print(train_x.shape)

In [ ]:
outliers = [4779, 4780,]
print("Removing {} outlying training data points".format(len(outliers)))
print(sorted(outliers))
train_x = np.delete(train_x, outliers, 0)
train_y = np.delete(train_y, outliers, 0)
train_l = np.delete(train_l, outliers)


## Normalization

In [ ]:
if normalize:
    means = []
    stds = []
    for band in tnrange(0, train_x.shape[-1]):
        mn = np.mean(train_x[..., band])
        std = np.std(train_x[..., band])
        normed = (train_x[..., band] - mn) / std
        normed[np.where(normed > 3)] = 3.
        normed[np.where(normed < -3)] = -3.
        train_x[..., band] = normed
        print(np.mean(train_x[..., band]))
        print(np.std(train_x[..., band]))
        means.append(mn)
        stds.append(std)

## Standardization

In [ ]:
# Until thanksgiving
min_all = [0.016768555792030362, 0.031859364002449986, 0.018888066882654337, 0.08300588143157157, 0.04900411763090337,
 0.07196114637821546, 0.0790915859490633, 0.08027978003529285,
 0.05206992316991091, 0.033226898834616644, 0.0,  0.0029407649199765626,
 -0.026773188909673434, 0.0032562207477167242, 0.006299999542534351, 4.999999873689376e-05]

max_all = [0.21292338743805878,
 0.29813900702779483, 0.4300807791948318, 0.5124803858995437,
 0.4764584118127815, 0.4890428495407102, 0.5156670875445272,
 0.5284151314082131,0.6588925588130947, 0.6016234779357905,
 0.37872035768296985, 0.7223387521306766, 0.40211242556571936,
 0.6531514992624433, 0.7853250503540039, 0.15389999747276306]

min_all = [0.016485823132097722, 0.03130804654210806, 0.018387083709239957,
           0.08212612159550192, 0.04826597087085247, 0.07179847471415998, 
           0.07879771515727044, 0.07925472483038905, 0.05206638704985381, 
           0.033287848345935345, 0.0, 7.580870587844813e-05, -0.027176744770258665,
           0.001232451893156402, 0.006309605669230223, 5.340657662600279e-05]
           
max_all = [0.21402996256947499, 0.2989177152514456, 0.43112194091081546, 
           0.512028709053993, 0.4776528477668758, 0.4895881682634349, 
           0.5156701475381851, 0.5278021097183228, 0.6594102591276167,
           0.6019508838653564, 0.36693188548088074, 0.7198386043310159, 0.40469192266464216,
           0.6503296375274656, 0.7188067436218262, 0.14778362214565277]

In [ ]:
if not normalize:
    #
    #min_all = []
    #max_all = []
    #
    for band in tnrange(0, train_x.shape[-1]):
        #mins = np.percentile(train_x[:-24, ..., band], 1)
        #maxs = np.percentile(train_x[:-24, ..., band], 99)
        mins = min_all[band]
        maxs = max_all[band]
        #print(mins1 - mins, maxs1 - maxs)
        train_x[..., band] = np.clip(train_x[..., band], mins, maxs)
        midrange = (maxs + mins) / 2
        rng = maxs - mins
        standardized = (train_x[..., band] - midrange) / (rng / 2)
        train_x[..., band] = standardized

        #min_all.append(mins)
        #max_all.append(maxs)

    print("The data has been scaled to [{}, {}]".format(np.min(train_x), np.max(train_x)))
    print(min_all, max_all)
    np.save("min_all.npy", min_all)
    np.save("max_all.npy", max_all)


# Augment training data

Horizontal and vertical flips for 4x augmentation.

**To do**
*  Random guassian noise
*  Brightness, contrast
*  Region swaps (randomply position positive samples at different locations)

## Load and process test data

In [ ]:
test_x = hkl.load("../tile_data/processed/test_x.hkl")
test_y = hkl.load("../tile_data/processed/test_y.hkl")
test_lengths = hkl.load("../tile_data/processed/test_lengths.hkl")
test_data = pd.read_csv("test_plot_ids.csv")
print(test_x.shape)
print(test_data.shape)

test_x = np.delete(test_x, 14, -1)

below_1 = [i for i, val in enumerate(test_x[..., :-2]) if np.min(val) < -1.66]
above_1 = [i for i, val in enumerate(test_x[..., :-2]) if np.max(val) > 1.66]
nans = [i for i, val in enumerate(test_x) if np.sum(np.isnan(val)) > 0]
outliers = below_1 + above_1 + nans
outliers = list(set(outliers))
print("There are {} outliers: {}".format(len(outliers), outliers))

test_x = np.delete(test_x, outliers, 0)
test_y = np.delete(test_y, outliers, 0)
test_lengths = np.delete(test_lengths, outliers, 0)
print([x for x in test_data['plot_id'].iloc[outliers]])
test_data = test_data.drop(outliers, 0)
test_data = test_data.reset_index(drop = True)

#for sample in tnrange(0, len(test_x)):
#    filtered = median_filter(test_x[sample, 0, :, :, 10], size = 5)
#    filtered = np.reshape(filtered, (8, 2, 8, 2))
#    filtered = np.mean(filtered, axis = (1, 3))
#    filtered = resize(filtered, (16, 16), 0)
#    test_x[sample, ..., 10] = np.stack([filtered] * 12)
    
#band_10 = test_x[..., 10]
#band_10[np.where(band_10 > 0.25)] = 0.25
#test_x[..., 10] = band_10

#test_x = np.reshape(test_x, (test_x.shape[0], 12, 2, 16, 16, 16))
#test_x = np.mean(test_x, axis = 2)
print(test_x.shape)
print(test_data.shape)

In [ ]:
test_x = hkl.load("../tile_data/processed/test_x.hkl")
test_y = hkl.load("../tile_data/processed/test_y.hkl")
test_lengths = hkl.load("../tile_data/processed/test_lengths.hkl")
test_data = pd.read_csv("test_plot_ids.csv")
print(test_x.shape)
print(test_data.shape)

test_x = np.delete(test_x, 14, -1)

In [ ]:
# Test normalization
if normalize:
    print("Normalizing data")
    for band in tnrange(0, test_x.shape[-1]):
        print(f"{band}, mean, {abs(np.mean(test_x[..., band]) - means[band])}")
        print(f"{band}, std, {abs(np.std(test_x[..., band]) - stds[band])}")
        normed = (test_x[..., band] - means[band]) / stds[band]
        normed[np.where(normed > 3)] = 3.
        normed[np.where(normed < -3)] = -3.
        test_x[..., band] = normed

        means.append(mn)
        stds.append(std)

In [ ]:
if not normalize:
    for band in range(0, test_x.shape[-1]):
        mins = min_all[band]
        maxs = max_all[band]
        test_x[..., band] = np.clip(test_x[..., band], mins, maxs)
        midrange = (maxs + mins) / 2
        rng = maxs - mins
        standardized = (test_x[..., band] - midrange) / (rng / 2)
        test_x[..., band] = standardized
    
    
    print("The data has been scaled to [{}, {}]".format(np.min(test_x), np.max(test_x)))


In [ ]:
print("Train and test characteristics:")
print("Train mean Y {}".format(np.mean([np.sum(x) for x in test_y])))
print("Test STD Y {}".format(np.std([np.sum(x) for x in test_y])))

print(f"There are {len(train_ids)} train and {len(test_ids)} test samples")
print("There is {} overlap between train and test".format(len([x for x in train_ids if x in test_ids])))

# Equibatch creation

The modelling approach uses equibatch sampling to ensure that there is a near constant standard deviation of the percent tree cover in the output labels for each batch. This helps ensure that the model performs equally well across gradients of tree cover, by mitigating the random possibility that many batches in a row near the end of sampling may be randomly biased towards a tree cover range.

In [ ]:
sums = np.sum(train_y, axis = (1, 2))
percents = [np.percentile(sums, x) for x in range(30, 100, 9)]
print(percents)
print("There are {} zeros".format(len(np.argwhere(sums == 0))))

In [ ]:
train_ids = [x for x in range(0, len(train_y))]

def multiplot(matrices):
    '''Plot multiple heatmaps with subplots
    
         Parameters:
          matrices (list of arrays):

         Returns:
          None
    '''
    fig, axs = plt.subplots(ncols=4)
    fig.set_size_inches(20, 4)
    for i, matrix in enumerate(matrices):
        sns.heatmap(data = matrix, ax = axs[i], vmin = 0, vmax = 0.9)
        axs[i].set_xlabel("")
        axs[i].set_ylabel("")
        axs[i].set_yticks([])
        axs[i].set_xticks([])

def equibatch(train_ids, p = percents):
    '''Docstring
    
         Parameters:
          train_ids (list):
          p (list):

         Returns:
          equibatches (list):
    '''
    percents = [9.0, 19.0, 29.0, 40.0, 64.0, 95.0, 145.0] # sept 28
    percents = [9.0, 19.0, 28.0, 40.0, 62.0, 100.0, 155.0] # november
   # percents =  [7.0, 13.0, 21.0, 33.0, 50.0, 80.0, 130.0] # overall
    #percents = [12.0, 24.0, 42.0, 65.0, 90.0, 125., 170.0] # lac
    #percents = [9.0, 19.0, 29.0, 40.0, 55.0, 85.0, 150.0] # east africa
    np.random.shuffle(train_ids)
    ix = train_ids
    percs = [np.sum(x) for x in train_y[ix]]
    ids0 = [x for x, z in zip(ix, percs) if z <= 2]
    ids30 = [x for x, z in zip(ix, percs) if 2 < z <= percents[0]]
    ids40 = [x for x, z in zip(ix, percs) if percents[0] < z <= percents[1]]
    ids50 = [x for x, z in zip(ix, percs) if percents[1] < z <= percents[2]]
    ids60 = [x for x, z in zip(ix, percs) if percents[2] < z <= percents[3]]
    ids70 = [x for x, z in zip(ix, percs) if percents[3] < z <= percents[4]]
    ids80 = [x for x, z in zip(ix, percs) if percents[4] < z <= percents[5]]
    ids90 = [x for x, z in zip(ix, percs) if percents[5] < z <= percents[6]]
    ids100 = [x for x, z in zip(ix, percs) if percents[6] < z]
    
    new_batches = []
    maxes = [len(ids0), len(ids30), len(ids40), len(ids50), len(ids60), len(ids70),
             len(ids80), len(ids90), len(ids100)]
    print(maxes)
    print(sum(maxes))
    cur_ids = [0] * len(maxes)
    iter_len = len(train_ids)//(len(maxes))
    for i in range(0, iter_len):
        for i, val in enumerate(cur_ids):
            if val > maxes[i] - 1:
                cur_ids[i] = 0
        if cur_ids[0] >= (maxes[0] - 2):
            cur_ids[0] = 0
        to_append = [ids0[cur_ids[0]],
                    ids30[cur_ids[1]], ids40[cur_ids[2]],
                    ids50[cur_ids[3]], ids60[cur_ids[4]], 
                    ids70[cur_ids[5]], ids80[cur_ids[6]],
                    ids90[cur_ids[7]], ids100[cur_ids[8]]]
        
        
        np.random.shuffle(to_append)
        new_batches.append(to_append)
        cur_ids = [x + 1 for x in cur_ids]
        
    new_batches = [item for sublist in new_batches for item in sublist]
    return new_batches

batch = equibatch(train_ids)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
f.set_size_inches(15, 6)
sns.distplot(np.sum(train_y, axis = (1, 2)), bins = 50, kde = False, ax = ax1)
ax1.set_title('Original distribution')
ax2.set_title('Equibatch distribution')
sns.distplot(np.sum(train_y[batch], axis = (1, 2)),
             bins = 50, kde = False, ax = ax2)

plt.show()

# Example equibatch

In [ ]:
multiplot([x.reshape((14, 14)) for x in train_y[batch[4:8]]])

In [ ]:
multiplot([x.reshape((14, 14)) for x in train_y[batch[8:12]]])

## Weight cross entropy by effective number of samples

In [ ]:
sum_pos = np.sum(train_y)
sum_neg = len(train_y) * 196 - sum_pos
print(sum_pos, sum_neg)
beta = 0.999
print("Beta: {}".format(beta))
samples_per_cls = np.array([sum_neg, sum_pos]) / 196
print(samples_per_cls)
effective_num = 1.0 - np.power(beta, samples_per_cls)
print(effective_num)
weights = (1.0 - beta) / np.array(effective_num)
weights = weights / np.sum(weights)
print("Neg and pos weights: {}".format(weights))
weight = weights[1] / weights[0]
print(weight)
weight = 1.7

In [ ]:
print("Baseline: The positive is: {}".format(weights[0]))
print("Baseline: The negative is: {}".format(weights[1]))
print("\n")
print("Balanced: The positive is: {}".format(weight*weights[0]))
print("Balanced: The negative is: {}".format(weights[1]))



# Loss definition

The current best loss is a combination of weighted binary cross entropy and per-image Lovasz-Softmax, with a loss schedule with the latter becoming more important each epoch

In [ ]:
from keras.losses import binary_crossentropy
import math
from scipy.ndimage import distance_transform_edt as distance

def calc_mask(seg):

    res = np.zeros_like(seg)
    posmask = seg.astype(np.bool)
    loss_importance = np.array([x for x in range(0, 197, 1)])
    loss_importance = loss_importance / 196
    loss_importance = np.expm1(loss_importance)
    loss_importance[:30] = 0.

    if posmask.any():
        negmask = ~posmask
        res = distance(negmask) * negmask - (distance(posmask) - 1) * posmask
        #res[np.where(res == 0)] = -1 * loss_importance[196 - sums]
    if np.sum(seg) == 196:
        res = np.ones_like(seg)
    if np.sum(seg) == 0:
        res = np.ones_like(seg)
    res[np.logical_and(res < 2, res > 0)] = 0.5
    res[np.logical_or(res >= 2, res <= 0)] = 1.
    return res# / max_value

def calc_mask_batch(y_true):
    '''Applies calc_dist_map to each sample in an input batch
    
         Parameters:
          y_true (arr):
          
         Returns:
          loss (arr):
    '''
    y_true_numpy = y_true.numpy()
    bce_batch = np.array([calc_mask(y)
                     for y in y_true_numpy]).astype(np.float32)
    return bce_batch

def weighted_bce_loss(y_true, y_pred, weight, mask = True, smooth = 0.03):
    '''Calculates the weighted binary cross entropy loss between y_true and
       y_pred with optional masking and smoothing for regularization
       
       For smoothing, we want to weight false positives as less important than
       false negatives, so we smooth false negatives 2x as much. 
    
         Parameters:
          y_true (arr):
          y_pred (arr):
          weight (float):
          mask (arr):
          smooth (float):

         Returns:
          loss (float):
    '''
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    y_true = K.clip(y_true, smooth, 1. - smooth)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    loss = tf.nn.weighted_cross_entropy_with_logits(
        y_true,
        logit_y_pred,
        weight,
    )

    return loss
    #return tf.reduce_mean(loss)
'''
    
def calc_dist_map(seg):
    #Utility function for calc_dist_map_batch that calculates the loss
     #  importance per pixel based on the surface distance function
    
    #     Parameters:
     #     seg (arr):
          
     #    Returns:
     #     res (arr):
    #
    res = np.zeros_like(seg)
    posmask = seg.astype(np.bool)
    loss_importance = np.array([x for x in range(0, 197, 1)])
    loss_importance = loss_importance / 196
    loss_importance = np.expm1(loss_importance)
    loss_importance[:30] = 0.

    if posmask.any():
        negmask = ~posmask
        res = distance(negmask) * negmask - (distance(posmask) - 1) * posmask
        sums = int(np.sum(seg))
        # When % = 1, 0 -> 1.75
        # When % = 100, 0 -> 0
        res[np.where(res == 0)] = -1 * loss_importance[196 - sums]
        res[np.where(np.isclose(res, -0.4142, rtol = 1e-2))] = -1 * loss_importance[196 - sums]
        # When % = 1, 1 -> 0
        # When % = 100, 1 -> 1.75
        res[np.where(res == 1)] = loss_importance[sums]
        res[np.where(np.isclose(res, 1.41421356, rtol = 1e-2))] = loss_importance[sums]
        
    res[np.where(res < -3)] = -3
    res[np.where(res > 3)] = 3
    if np.sum(seg) == 196:
        res = np.ones_like(seg)
        res *= -1
    if np.sum(seg) == 0:
        res = np.ones_like(seg)
    return res# / max_value
'''

def calc_dist_map(seg):
    #Utility function for calc_dist_map_batch that calculates the loss
    #   importance per pixel based on the surface distance function
    
     #    Parameters:
    #      seg (arr):
     #     
    #     Returns:
    #      res (arr):
    #
    res = np.zeros_like(seg)
    posmask = seg.astype(np.bool)
    loss_importance = np.array([x for x in range(0, 197, 1)])
    loss_importance = loss_importance / 196
    loss_importance[:20] = 0.
    loss_importance[20:60] = np.arange(0, 40, 1) / 40
    loss_importance[60:] = 1.

    mults = np.ones_like(seg)
    ones = np.ones_like(seg)
    for x in range(1, res.shape[0] -1 ):
        for y in range(1, res.shape[0] - 1):
            if seg[x, y] == 1:
                l = seg[x - 1, y]
                r = seg[x + 1, y]
                u = seg[x, y + 1]
                d = seg[x, y - 1]
                lu = seg[x - 1, y + 1]
                ru = seg[x + 1, y + 1]
                rd = seg[x + 1, y - 1]
                ld = seg[x -1, y - 1]
                
                sums = (l + r + u + d)
                sums2 = (l + r + u + d + lu + ru +rd + ld)
                if sums >= 2:
                    mults[x, y] = 1.5
                if sums2 <= 1:
                    ones[x - 1, y] = 0.25
                    ones[x + 1, y] = 0.25
                    ones[x, y + 1] = 0.25
                    ones[x, y - 1] = 0.25
                    ones[x - 1, y + 1] = 0.25
                    ones[x + 1, y + 1] = 0.25
                    ones[x + 1, y - 1] = 0.25
                    ones[x -1, y - 1] = 0.25

    if posmask.any():
        
        negmask = ~posmask
        res = distance(negmask) * negmask - (distance(posmask) - 1) * posmask
        #sums = int(np.sum(seg))
        # When % = 1, 0 -> 1.75
        # When % = 100, 0 -> 0
        res = np.round(res, 0)
        res[np.where(np.isclose(res, -.41421356, rtol = 1e-2))] = -1
        res[np.where(res == -1)] = -1 * mults[np.where(res == -1)]
        res[np.where(res == 0)] = -1  * mults[np.where(res == 0)]# * loss_importance[196 - sums]
        # When % = 1, 1 -> 0
        # When % = 100, 1 -> 1.75
        res[np.where(res == 1)] = 1 * ones[np.where(res == 1)]
        res[np.where(res == 1)] *= 0.67
        #res[np.where(np.isclose(res, 1.41421356, rtol = 1e-2))] = loss_importance[sums]
        
    res[np.where(res < -3)] = -3
    res[np.where(res > 3)] = 3
    if np.sum(seg) == 196:
        res = np.ones_like(seg)
        res *= -1
    if np.sum(seg) == 0:
        res = np.ones_like(seg)
    return res# / max_value


def calc_lovasz_weight(y_true):
    if np.sum(y_true) > 12:
        return np.array(1., dtype = np.float32)
    else:
        return np.array(0., dtype = np.float32)
    
def calc_bce_weight(y_true):
    if np.sum(y_true) <= 12:
        return np.array(1., dtype = np.float32)
    else:
        return np.array(0., dtype = np.float32)
    
def calc_lovasz_weight_batch(y_true):
    '''Applies calc_dist_map to each sample in an input batch
    
         Parameters:
          y_true (arr):
          
         Returns:
          loss (arr):
    '''
    y_true_numpy = y_true.numpy()
    lovasz_batch = np.array([calc_lovasz_weight(y)
                     for y in y_true_numpy]).astype(np.float32)
    return lovasz_batch

def calc_bce_weight_batch(y_true):
    '''Applies calc_dist_map to each sample in an input batch
    
         Parameters:
          y_true (arr):
          
         Returns:
          loss (arr):
    '''
    y_true_numpy = y_true.numpy()
    bce_batch = np.array([calc_bce_weight(y)
                     for y in y_true_numpy]).astype(np.float32)
    return bce_batch
    
    

def calc_dist_map_batch(y_true):
    '''Applies calc_dist_map to each sample in an input batch
    
         Parameters:
          y_true (arr):
          
         Returns:
          loss (arr):
    '''
    y_true_numpy = y_true.numpy()
    return np.array([calc_dist_map(y)
                     for y in y_true_numpy]).astype(np.float32)

def surface_loss(y_true, y_pred):
    '''Calculates the mean surface loss for the input batch
       by multiplying the distance map by y_pred
    
         Parameters:
          y_true (arr):
          y_pred (arr):
          
         Returns:
          loss (arr):
        
         References:
          https://arxiv.org/abs/1812.07032
    '''
    y_true_dist_map = tf.py_function(func=calc_dist_map_batch,
                                     inp=[y_true],
                                     Tout=tf.float32)
    y_true_dist_map = tf.stack(y_true_dist_map, axis = 0)
    multipled = y_pred * y_true_dist_map
    #loss = tf.reduce_mean(multipled, axis = (1, 2, 3))
    loss = K.mean(multipled)
    return loss



def lovasz_surf(y_true, y_pred, alpha, weight, beta):
    
    #lv = lovasz_softmax(probas = y_pred,
    #                    labels = tf.reshape(y_true, (-1, 14, 14)), 
    #                    classes=[1],
    #                    per_image=True) 
    #print("Lovasz Shape: ", lv.shape)
    
    bce = weighted_bce_loss(y_true = y_true, 
                             y_pred = y_pred, 
                             weight = weight,
                             smooth = 0.03)
    #mask_wt = tf.py_function(func=calc_mask_batch,
    #                                 inp=[y_true],
    #                                 Tout=tf.float32)
   # bce = bce * mask_wt
    #print("BCE Shape: ", bce.shape)
    #return tf.reduce_mean(bce)

    
    bce = tf.reduce_mean(bce, axis = (1, 2, 3))
    #print("BCE Shape: ", bce.shape)
    surface = surface_loss(y_true, y_pred)
    
    #lv_mask = tf.math.reduce_sum(y_true, axis = (1, 2, 3))
    #lv_mask = tf.cast(lv_mask, tf.float32)
    #lv_mask = tf.math.greater_equal(lv_mask, tf.constant([25.]))
    #lv_mask = tf.cast(lv_mask, tf.float32)

    
    #bce_mask = tf.math.reduce_sum(y_true, axis = (1, 2, 3))
    #bce_mask = tf.cast(bce_mask, tf.float32)
    #bce_mask = tf.math.less(bce_mask, tf.constant([25.]))
    #bce_mask = tf.cast(bce_mask, tf.float32)
    
    
    #lovasz = (lv * lv_mask) + (bce * bce_mask)
    #lovasz = tf.reduce_mean(lovasz)
    bce = tf.reduce_mean(bce)
    

    #lovasz_portion = (1 - alpha) * lovasz
    #lovasz_portion = lovasz_portion * beta
    bce = (1 - alpha) * bce
    #bce = bce * (1 - beta)
    surface_portion = alpha * surface
    

    
    result = bce + surface_portion
    return result


In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
f.set_size_inches(14, 5)
sns.heatmap(calc_mask(train_y[-4]), ax = ax1)
ax2.set_title('Ground truth Y')
ax1.set_title('Boundary loss mask')
sns.heatmap(train_y[-4], ax = ax2)
plt.show()

# Evaluation metrics

In [ ]:
countries = {'all': [0, 1150]}

def dice_loss_tolerance(y_true, y_pred):
    numerator_data = np.zeros_like(y_true)
    for x in range(y_true.shape[0]):
        for y in range(y_true.shape[1]):
            min_x = np.max([0, x-1])
            min_y = np.max([0, y-1])
            max_y = np.min([y_true.shape[0], y+2])
            max_x = np.min([y_true.shape[0], x+2])
            if y_true[x, y] == 1:
                numerator_data[x, y] = np.max(y_pred[min_x:max_x, min_y:max_y])
                
    numerator = 2 * np.sum(y_true * numerator_data, axis=-1)
    denominator = np.sum(y_true + y_pred, axis=-1)
    return (numerator + 1) / (denominator + 1)
                    
            
        
def compute_f1_score_at_tolerance(true, pred, tolerance = 1):
    fp = 0
    tp = 0
    fn = 0
    
    tp = np.zeros_like(true)
    fp = np.zeros_like(true)
    fn = np.zeros_like(true)
    
    
    for x in range(true.shape[0]):
        for y in range(true.shape[1]):
            min_x = np.max([0, x-1])
            min_y = np.max([0, y-1])
            max_y = np.min([true.shape[0], y+2])
            max_x = np.min([true.shape[0], x+2])
            if true[x, y] == 1:
                if np.sum(pred[min_x:max_x, min_y:max_y]) > 0:
                    tp[x, y] = 1
                else:
                    fn[x, y] = 1
            if pred[x, y] == 1:
                if np.sum(true[min_x:max_x, min_y:max_y]) > 0:
                    if true[x, y] == 1:
                        tp[x, y] = 1
                else:
                    fp[x, y] = 1                
                
    return np.sum(tp), np.sum(fp), np.sum(fn)

def recover_patches(arr, thresh_p):
    sum1 = np.sum(arr)
    thresh = thresh_p / 2

    for window_x in range(2, arr.shape[0]-2, 1):
        for window_y in range(2, arr.shape[1]-2, 1):
            l, r, u, d =  False, False, False, False
            cur_window = arr[window_x-2:window_x+3, window_y-2:window_y+3]
            hor_vert_neighbors = False
            if (cur_window[2, 2] > thresh and
                cur_window[2, 2] < thresh_p):
                if cur_window[3, 2] > thresh * 2 and cur_window[4, 2] < thresh * 2 :
                    hor_vert_neighbors = True
                    r = True
                if cur_window[1, 2] > thresh * 2 and cur_window[2, 0] < thresh * 2:
                    hor_vert_neighbors = True
                    l = True
                if cur_window[2, 3] > thresh * 2 and cur_window[2, 4] < thresh * 2:
                    hor_vert_neighbors = True
                    d = True
                if cur_window[2, 1] > thresh * 2 and cur_window[2, 0] < thresh * 2:
                    hor_vert_neighbors = True
                    u = True
            passes = False
            if r and not l:
                passes = True
            if l and not r:
                passes = True
            if d and not u:
                passes = True
            if u and not d:
                passes = True

            if passes:
                if r:
                    if cur_window[1, 1] < thresh * 2 and cur_window[1, 3] < thresh * 2:
                        arr[window_x, window_y] = (thresh_p + 0.01)

                if l:
                    if cur_window[3, 1] < thresh * 2 and cur_window[3, 3] < thresh * 2:
                        arr[window_x, window_y] = (thresh_p + 0.01)

                if u:
                    if cur_window[1, 3] < thresh * 2 and cur_window[3, 3] < thresh * 2:
                        arr[window_x, window_y] = (thresh_p + 0.01)

                if d:
                    if cur_window[1, 1] < thresh * 2 and cur_window[3, 1] < thresh * 2:
                        arr[window_x, window_y] = (thresh_p + 0.01)
    return arr


def calculate_metrics(country, al = 0.4, canopy_thresh = 100):
    '''Calculates the following metrics for an input country, based on
       indexing of the country dictionary:
       
         - Loss
         - F1
         - Precision
         - Recall
         - Dice
         - Mean surface distance
         - Average error
    
         Parameters:
          country (str):
          al (float):
          
         Returns:
          val_loss (float):
          best_dice (float):
          error (float):
    '''
    print(canopy_thresh)
    start_idx = 0
    stop_idx = len(test_x)
    best_f1 = 0
    best_dice = 0
    best_thresh = 0
    hausdorff = 0
    relaxed_f1 = 0
    preds = []
    vls = []
    trues = []
    test_ids = [x for x in range(len(test_x))]
    for test_sample in test_ids[start_idx:stop_idx]:
        if np.sum(test_y[test_sample]) < ((canopy_thresh/100) * 197):
            x_input = test_x[test_sample].reshape(1, 12, 16, 16, n_bands)
            x_median_input = calc_median_input(x_input)
            y, vl = sess.run([fm, test_loss], feed_dict={inp: x_input,
                                                         inp_median: x_median_input,
                                                          length: np.full((1, 1), 12),
                                                          is_training: False,
                                                          clipping_params['rmax']: rmax_epoch,
                                                          clipping_params['rmin']: rmin_epoch,
                                                          clipping_params['dmax']: dmax_epoch,
                                                          labels: test_y[test_sample].reshape(1, 14, 14),
                                                          loss_weight: 1.7,
                                                          alpha: al,
                                                          })
            preds.append(y.reshape((14, 14)))
            vls.append(vl)
            trues.append(test_y[test_sample].reshape((14, 14)))
    dice_losses = []
    for thresh in range(8, 10):
        tps_relaxed = np.empty((len(preds), ))
        fps_relaxed = np.empty((len(preds), ))
        fns_relaxed = np.empty((len(preds), ))
        abs_error = np.empty((len(preds), ))
        
        for sample in range(len(preds)):
            pred = np.copy(preds[sample])
            true = trues[sample]
            if thresh == 8:
                if np.sum(true + pred) > 0:
                    dice_losses.append(dice_loss_tolerance(np.array(true), np.array(pred)))
                else:
                    dice_losses.append(1.)
            pred[np.where(pred >= thresh*0.05)] = 1
            pred[np.where(pred < thresh*0.05)] = 0
            
            true_s = np.sum(true[1:-1])
            pred_s = np.sum(pred[1:-1])
            abs_error[sample] = abs(true_s - pred_s)
            tp_relaxed, fp_relaxed, fn_relaxed = compute_f1_score_at_tolerance(true, pred)
            tps_relaxed[sample] = tp_relaxed
            fps_relaxed[sample] = fp_relaxed
            fns_relaxed[sample] = fn_relaxed                   
            
        oa_error = np.mean(abs_error)
        precision_r = np.sum(tps_relaxed) / (np.sum(tps_relaxed) + np.sum(fps_relaxed))
        recall_r = np.sum(tps_relaxed) / (np.sum(tps_relaxed) + np.sum(fns_relaxed))
        f1_r = 2*((precision_r* recall_r) / (precision_r + recall_r))
        
        if f1_r > best_f1:
            haus = np.zeros((len(preds), ))
            for sample in range(len(preds)):
                pred = np.copy(preds[sample])
                pred[np.where(pred >= thresh*0.05)] = 1
                pred[np.where(pred < thresh*0.05)] = 0
                true = trues[sample]
                dists = compute_surface_distances(np.array(true).reshape(14, 14, 1).astype(int),
                                                  np.array(pred).reshape(14, 14, 1).astype(int),
                                                  [1, 1, 1])
                if np.sum(true + pred) > 0:
                    haus_i = compute_robust_hausdorff(dists, 50)
                    if not np.isinf(haus_i):
                        haus[sample] = haus_i
                if np.sum(true + pred) == 0:
                    haus[sample] = 0.
                    
            dices = np.mean(dice_losses)
            haus = np.mean(haus)
            best_dice = dices
            best_f1 = f1_r
            p = precision_r
            r = recall_r
            error = oa_error
            best_thresh = thresh*0.05
            best_haus = haus
    print("{}: Val loss: {} Thresh: {} F1: {}"
          " R: {} P: {} D: {} H: {} Error: {}".format(country, 
                                                     np.around(np.mean(vls), 3),
                                                     np.around(best_thresh, 2),
                                                     np.around(best_f1, 3), np.around(p, 3),
                                                     np.around(r, 3), 
                                                     np.around(np.mean(best_dice), 3),
                                                     np.around(best_haus, 3),
                                                     np.around(error, 3)))
    return np.mean(vls), best_f1, error, best_haus, np.mean(best_dice)

# Graph set up

In [ ]:
def augment_batch(batch_ids, batch_size):
    '''Performs random flips and rotations of the X and Y
       data for a total of 4 x augmentation
    
         Parameters:
          batch_ids (list):
          batch_size (int):
          
         Returns:
          x_batch (arr):
          y_batch (arr):
    '''
    x = train_x[batch_ids]
    y = train_y[batch_ids]
    x_batch = np.zeros_like(x)
    y_batch = np.zeros_like(y)
    flips = np.random.choice(np.array([0, 1, 2, 3]), batch_size, replace = True)
    for i in range(x.shape[0]):
        current_flip = flips[i]
        if current_flip == 0:
            x_batch[i] = x[i]
            y_batch[i] = y[i]
        if current_flip == 1:
            x_batch[i] = np.flip(x[i], 1)
            y_batch[i] = np.flip(y[i], 0)
        if current_flip == 2:
            x_batch[i] = np.flip(x[i], [2, 1])
            y_batch[i] = np.flip(y[i], [1, 0])
        if current_flip == 3:
            x_batch[i] = np.flip(x[i], 2)
            y_batch[i] = np.flip(y[i], 1)

    y_batch = y_batch.reshape((batch_size, 14, 14))
    return x_batch, y_batch

x_batch_test, y_batch_test = augment_batch([0,], 1)


In [ ]:
def calc_median_input(x_batch):
    x_median = np.median(x_batch, axis = (1))
    return x_median

x_batch_med = calc_median_input(x_batch_test)

In [ ]:
FRESH_START = True
best_val = 0.2

START_EPOCH = 1
END_EPOCH = 100
LEARNING_RATE = 3e-3
test_ids = [x for x in range(0, len(test_x))]

print(f"Starting model with: \n {ZONE_OUT_PROB} zone out \n {L2_REG} l2 \n"
      f"{INITIAL_LR} initial LR \n {FINAL_LR} final LR \n {total_parameters} parameters")        
    
if FRESH_START:
    print(f"Restarting training from scratch on {len(train_ids)} train and {len(test_ids)} test samples")
    #optimizer = AdaBoundOptimizer(1e-4, 7e-3)
    optimizer = tf.train.AdamOptimizer(5e-5)
    train_loss = lovasz_surf(tf.reshape(labels, (-1, 14, 14, 1)), 
                             fm, weight = loss_weight, 
                             alpha = alpha, beta = beta_)
    #l2_loss = tf.losses.get_regularization_l05oss()
    #train_loss += l2_loss
#
    #ft_optimizer = tf.train.GradientDescentOptimizer(ft_lr)
    ft_optimizer = tf.train.MomentumOptimizer(ft_lr, momentum = 0.8, use_nesterov = True)
    test_loss = lovasz_surf(tf.reshape(labels, (-1, 14, 14, 1)),
                            fm, weight = loss_weight, 
                            alpha = alpha, beta = beta_)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(train_loss)   
        ft_op = ft_optimizer.minimize(train_loss)
        
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    saver = tf.train.Saver(max_to_keep = 150)
    
print("The graph has been finalized")

In [ ]:
model_path  = "../models/unet-adam-5e-5/"
if os.path.isfile(f"{model_path}metrics.npy"):
    metrics = np.load(f"{model_path}metrics.npy")
    print(f"Loading {model_path}metrics.npy")
else:
    metrics = np.zeros((6, 300))

In [ ]:
# This cell should be run to do fine-tuning, if commented - train from scratch
#@path = '../models/tropics/2e-5/ft-bce-surf/322-92-0/'
path = "../models/unet-adam-5e-5/190-90-3/"
#path = "../models/unet-new-dem/145-88-4/"
#path = "../models/unet-s1/9-74-4/"

#path = '../models/nov-unet/85-86-6/'
#new_saver = tf.train.import_meta_graph(path + 'model.meta')
saver.restore(sess, tf.train.latest_checkpoint(path))


In [ ]:
start = 150
end = 170

f, ((c1r1, c1r2, c1r3), (c2r1, c2r2, c2r3)) = plt.subplots(2, 3, sharey=False)
f.set_size_inches(15, 12)

c1r1.set_title("Train losses - 0.33 a, 1.7 w, 1e-4, 7e-3 ")
sns.scatterplot(y = metrics[0, start:end], x = np.arange(start, end),
               ax = c1r1)

c1r2.set_title("Nov 25 - F1 score")
sns.scatterplot(y = metrics[5, start:end], x = np.arange(start, end),
               ax = c1r2)

c2r1.set_title("Test losses")
sns.scatterplot(y = metrics[1, start:end], x = np.arange(start, end),
               ax = c2r1)

c2r2.set_title("Errors")
sns.scatterplot(y = metrics[2, start:end], x = np.arange(start, end),
               ax = c2r2)

c2r3.set_title("Hausdorff")
sns.scatterplot(y = metrics[3, start:end], x = np.arange(start, end),
               ax = c2r3)

c1r3.set_title("Dice")
sns.scatterplot(y = metrics[4, start:end], x = np.arange(start, end),
               ax = c1r3)
plt.show()

In [ ]:
# Make sure that the equibatch is working with the augmentation
randomize = equibatch(train_ids)
sum_no_equibatch = []
sum_equibatch = []
for k in tnrange(int(len(randomize) // 20)):
    rand = [x for x in range(len(randomize))]
    batch_ids = rand[k*BATCH_SIZE:(k+1)*BATCH_SIZE]
    _, y_batch = augment_batch(batch_ids, BATCH_SIZE)
    sum_no_equibatch.append(np.sum(y_batch))
    
for k in tnrange(int(len(randomize) // 20)):
    batch_ids = randomize[k*BATCH_SIZE:(k+1)*BATCH_SIZE]
    _, y_batch = augment_batch(batch_ids, BATCH_SIZE)
    sum_equibatch.append(np.sum(y_batch))
    
print("No equibatch SD: {}".format(np.std(np.array(sum_no_equibatch))))
print("Equibatch SD: {}".format(np.std(np.array(sum_equibatch))))

# September changes
- Implement equibatch
- Implement 4x4 FPA, with CSSE in middle blocks
- Reduce label smoothing from 0.08 to 0.03
- reduce dropblock to 0.95 from 0.8

# Things to test
- Concatenating mean after GRU
- Turning of sse in GRU

In [ ]:
# import scipy
best_val = 0.72
fine_tune = True
countries['all'] = [0, len(test_x)]
ft_epochs = 0
ft_learning_rate = 5e-4



ft_e = 0
for i in range(189, 250):
    ft_e += 1
    al = np.min( [0.01 * (i - 1), 0.33] )
    be = 0.0
    test_al = 0.33 if al < 0.8 else al
    print(ft_epochs)
    if fine_tune == True:
        op = ft_op
        if ft_epochs >= 10:
            ft_learning_rate /= 2
            ft_epochs = 0
            print("DIVIDING LR", ft_learning_rate)
        #ft_epochs += 1
        #if ft_epochs % 20 == 0:
        #    ft_learning_rate /= 2
        #    print("DIVIDING LR")
        print(f"FINE TUNING WITH {ft_learning_rate} LR")
    else:
        op = op = train_op
        
    train_ids = [x for x in range(len(train_y))]
    randomize = equibatch(train_ids)
    print(f"starting epoch {i}, alpha: {al}, beta: {be} drop: {np.max((1. - (i*0.0025), 0.85))}")
    
    loss = train_loss
    BATCH_SIZE = 20
    test_ids = [x for x in range(0, len(test_x))]
    losses = []
    
    for k in tnrange(int(len(randomize) // BATCH_SIZE)):
        rmax_epoch, dmax_epoch, rmin_epoch = calc_renorm_params(i, len(train_y), 20, k)
        if k % 8 == 0:
            sleep(1.9)
        batch_ids = randomize[k*BATCH_SIZE:(k+1)*BATCH_SIZE]
        if any([x for x in batch_ids if x in np.arange(len(train_ids) - 16, len(train_ids))]):
            print("New sample")
        x_batch, y_batch = augment_batch(batch_ids, BATCH_SIZE)
        x_median_input = calc_median_input(x_batch)
        opt, tr = sess.run([op, loss],
                          feed_dict={inp: x_batch,
                                     inp_median: x_median_input,
                                     length: np.full((BATCH_SIZE, 1), 12),
                                     labels: y_batch,
                                     is_training: True,
                                     clipping_params['rmax']: rmax_epoch,
                                     clipping_params['rmin']: rmin_epoch,
                                     clipping_params['dmax']: dmax_epoch,
                                     loss_weight: 1.7,
                                     keep_rate: np.max((1. - (i * 0.0025), 0.875)),
                                     alpha: al,
                                     beta_: be,
                                     ft_lr: ft_learning_rate,
                                     })
        losses.append(tr)
    
    print(f"Epoch {i}: Loss {np.around(np.mean(losses[:-1]), 3)}")
    val_loss, f1, error, haus, dice = calculate_metrics('all', al = test_al, canopy_thresh = 75)
    metrics[0, i] = np.mean(losses[:-1])
    metrics[1, i] = val_loss
    metrics[2, i] = error
    metrics[3, i] = haus
    metrics[4, i] = dice
    metrics[5, i] = f1
    if f1 < (best_val - 0.002):
        ft_epochs += 1
    if f1 > (best_val - 0.02):
        print(f"Saving model with {f1}")
        np.save(f"{model_path}metrics.npy", metrics)
        os.mkdir(f"{model_path}{str(i)}-{str(f1*100)[:2]}-{str(f1*100)[3]}/")
        save_path = saver.save(sess, f"{model_path}{str(i)}-{str(f1*100)[:2]}-{str(f1*100)[3]}/model")
        if f1 > best_val:
            best_val = f1

In [ ]:
i = 188
f1 = 0.901
os.mkdir(f"{model_path}{str(i)}-{str(f1*100)[:2]}-{str(f1*100)[3]}/")
save_path = saver.save(sess, f"{model_path}{str(i)}-{str(f1*100)[:2]}-{str(f1*100)[3]}/model")

In [ ]:
test_ids = [x for x in range(len(test_x))]
val_loss, f1, error, haus, dice = calculate_metrics('all', al = test_al, canopy_thresh = 100)

In [ ]:
test_ids = [x for x in range(len(test_x))]
val_loss, f1, error, haus, dice = calculate_metrics('all', al = test_al, canopy_thresh = 100)

In [ ]:
# 12 time step, 16, 8, 6, 4, 8, 16, 14 UNET 
#Epoch 100: Loss 0.25600001215934753
#all: Val loss: 0.2709 Thresh: 0.5 F1: 0.878 R: 0.9 P: 0.857 D: 0.8044324218159632 H: 0.511 Error: 8.829

# 12 time step, ibid, no sse in convGRU
#Epoch 67: Loss 0.26600000262260437
#all: Val loss: 0.2750 Thresh: 0.4 F1: 0.87 R: 0.882 P: 0.858 D: 0.717465778 H: 0.559 Error: 10.59


# Mean concat
#Epoch 51: Loss 0.2709999978542328
# all: Val loss: 0.272000 Thresh: 0.45 F1: 0.878 R: 0.899 P: 0.857 D: 0.764088 H: 0.584 Error: 9.635

# Avbove, 5e-3
# Epoch 165: Loss 0.24699999392032623
#all: Val loss: 0.2590 Thresh: 0.4 F1: 0.887 R: 0.91 P: 0.866 D: 0.7396746 H: 0.499 Error: 7.854
# Updated test data
#all: Val loss: 0.2560 Thresh: 0.4 F1: 0.89 R: 0.917 P: 0.866 D: 0.6593579 H: 0.453 Error: 7.809

# Fixed sentinel 1 , epoch 129
#all: Val loss: 0.257999986410141 Thresh: 0.4 F1: 0.893 R: 0.92 P: 0.869 D: 0.651 H: 0.437 Error: 8.401

# ADAM, 0.33 Alpha
# all: Val loss: 0.24799999594688416 Thresh: 0.45 F1: 0.904 R: 0.932 P: 0.878 D: 0.672 H: 0.418 Error: 7.191

# Model validation and sanity checks





In [ ]:
preds2 = np.concatenate(preds).flatten()
trues2 = np.concatenate(trues).flatten()
tp = preds2 * trues2
fn = [1 if x > y else 0 for (x, y) in zip(trues2, preds2)]
fp = [1 if y > x else 0 for (x, y) in zip(trues2, preds2)]
tn = (len(test_y) * 196) - np.sum(tp) - np.sum(fn) - np.sum(fp)
print("TP {}, FN {}, FP {}, TN {}".format(np.sum(tp), sum(fn), np.sum(fp), tn))
print(len(preds2))

In [ ]:
tps = []
fns = []
fps = []
for i in range(0, len(test_y)*196, 196):
    tps.append(np.sum(tp[i:i+196]))
    fns.append(np.sum(fn[i:i+196]))
    fps.append(np.sum(fp[i:i+196]))

In [ ]:
from sklearn.metrics import f1_score

true_canopy = []
error_canopy = []
pred_canopy = []
f1_hard = []
tp_softs = []
fp_softs = []
fn_softs = []
for i in range(len(trues)):
    true_canopy.append(np.sum(trues[i]) / 1.96)
    error_canopy.append(abs(np.sum(preds[i]) - np.sum(trues[i])) / 1.96)
    pred_canopy.append(np.sum(preds[i]) / 1.96)
    f1_hard.append(f1_score(trues[i], preds[i]))
    tp_soft, fp_soft, fn_soft = compute_f1_score_at_tolerance(np.array(trues[i].reshape((14, 14))),
                                                 np.array(preds[i].reshape((14, 14))))
    tp_softs.append(tp_soft)
    fp_softs.append(fp_soft)
    fn_softs.append(fn_soft)

In [ ]:
precision = np.sum(tp_softs) / (np.sum(tp_softs) + np.sum(fp_softs))
recall = np.sum(tp_softs) / (np.sum(tp_softs) + np.sum(fn_softs))
print(precision, recall, np.mean(error_canopy), np.mean(true_canopy), np.mean(pred_canopy))

In [ ]:
plot_ids_loaded = np.load("../data/metrics/plotids.npy")
plot_ids_loaded = np.delete(plot_ids_loaded, outliers, 0)

In [ ]:
metrics = pd.DataFrame({'true': true_canopy,
                        'pred': pred_canopy,
                        'f1_hard': f1_hard,
                        'error': error_canopy,
                        'tp': tps,
                        'fp': fps,
                        'fn': fns,
                        'tp_soft': tp_softs,
                        'fp_soft': fp_softs,
                        'fn_soft': fn_softs,
                       })

res = map(lambda x: int(math.floor(np.min([x, 90]) / 10.0)) * 10, true_canopy)
res = [x for x in res]
metrics['group'] = res
metrics['model'] = 'proposed'
#metrics['slope'] = slopes
metrics['plot_id'] = plot_ids_loaded
metrics.to_csv("../data/metrics/proposed-sample.csv")

In [ ]:
metrics = pd.read_csv("../data/metrics/proposed-sample.csv")
continents = pd.read_csv("../data/latlongs/test_continents.csv")
continents = continents.join(metrics, how = 'inner')
continents.groupby("CONTINENT").mean()

In [ ]:
hard_precisions = metrics.groupby('group').apply(lambda x: (np.sum(x.tp) / np.sum(x.tp + x.fp)))
hard_recalls = metrics.groupby('group').apply(lambda x: (np.sum(x.tp) / np.sum(x.tp + x.fn)))
errors = metrics.groupby('group').apply(lambda x: np.mean(x.error))
hard_f1 = 2 *  ((hard_precisions * hard_recalls) / (hard_precisions + hard_recalls))

precisions = metrics.groupby('group').apply(lambda x: (np.sum(x.tp_soft) / np.sum(x.tp_soft + x.fp_soft)))
recalls = metrics.groupby('group').apply(lambda x: (np.sum(x.tp_soft) / np.sum(x.tp_soft + x.fn_soft)))
soft_f1 = 2 *  ((precisions * recalls) / (precisions + recalls))

In [ ]:
new_metrics = pd.DataFrame({'group': [x for x in range(0, 100, 10)],
                            'hard_rec': hard_recalls,
                            'soft_rec': recalls,
                            'hard_prec': hard_precisions,
                            'soft_prec': precisions,
                            'hard_f1': hard_f1,
                            'soft_f1': soft_f1,
                            'error': errors,
                            'model': 'proposed'
                           })

#new_metrics.to_csv("../data/metrics/proposed.csv")

# Prediction visualizations

In [ ]:
test_ids = [x for x in range(len(test_x))]
diffs = []
for idx in tnrange(len(test_ids)):
    x_input = test_x[idx].reshape(1, 12, IMAGE_SIZE, IMAGE_SIZE, n_bands)
    median_input = calc_median_input(x_input)
    y = sess.run([fm], feed_dict={inp: x_input,
                                  inp_median: median_input,
                                  length: np.full((1, 1), 12),
                                  is_training: False,
                                  clipping_params['rmax']: rmax_epoch,
                                  clipping_params['rmin']: rmin_epoch,
                                  clipping_params['dmax']: dmax_epoch,
                                  })
    y = np.array(y).reshape(14, 14)
    y[np.where(y > 0.4)] = 1.0
    y[np.where(y < 0.4)] = 0.
    diff = np.sum(y) - np.sum(test_y[idx])
    diffs.append(diff)

In [ ]:
start = 0


test_ids = [x for x in range(0, len(test_x))]

def multiplot(matrices, nrows = 2, ncols = 4):
    '''Docstring
    
         Parameters:
          matrices (list):
          nrows (int):
          
         Returns:
          None
    '''
    fig, axs = plt.subplots(ncols=4, nrows = nrows)
    fig.set_size_inches(20, 4*nrows)
    to_iter = [[x for x in range(i, i + ncols + 1)] for i in range(0, nrows*ncols, ncols)]
    for r in range(1, nrows + 1):
        min_i = min(to_iter[r-1])
        max_i = max(to_iter[r-1])
        for i, matrix in enumerate(matrices[min_i:max_i]):
            sns.heatmap(data = matrix, ax = axs[r - 1, i], vmin = 0, vmax = 0.9)
            axs[r - 1, i].set_xlabel("")
            axs[r - 1, i].set_ylabel("")
            axs[r - 1, i].set_yticks([])
            axs[r - 1, i].set_xticks([])
    plt.show



In [ ]:
def identify_alignment(true, pred, wsize = 3, difference = 0.2):
    '''Docstring
    
         Parameters:
          true (arr):
          pred (arr):
          wsize (int):
          difference (float):
          
         Returns:
          None
    '''
    n_single_trees = 0
    for x in range(1, true.shape[0] - 1, 1):
        for y in range(1, true.shape[1] - 1, 1):
            wind_true = true[x-1:x+2, y-1:y+2]
            wind_pred = pred[x-1:x+2, y-1:y+2]
            if wind_true[1, 1] == 1:
                if np.sum(wind_true) == 1:
                    n_single_trees += 1
                    pred_place = np.argmax(wind_pred.flatten())
                    diff = wind_pred.flatten()[pred_place] - wind_pred.flatten()[4]
                    if pred_place != 4:
                        if diff > difference:
                            x_lv = pred_place // 3
                            y_lv = pred_place % 3
                            print(x_lv, y_lv)
                            proposed = wind_true[x_lv - 1:x_lv+2, y_lv-1:y_lv+2]
                            if np.sum(proposed) == 0:
                                print("There is a missed position at {} x, {} y: {}".format(x, y, diff))

In [ ]:
def aggregate_maxes(y_true, y_pred):
    y_out = np.copy(y_pred)
    change_map = np.zeros((14, 14))
    for x in range(y_true.shape[0]):
        for y in range(y_true.shape[0]):
            min_x = np.max([0, x-1])
            min_y = np.max([0, y-1])
            max_y = np.min([y_true.shape[1], y+2])
            max_x = np.min([y_true.shape[1], x+2])
            
            y_true_sum = np.sum(y_true[min_x:max_x, min_y:max_y])
            y_pred_max = np.max(y_pred[min_x:max_x, min_y:max_y])
            y_pred_min = np.min(y_pred[min_x:max_x, min_y:max_y])
            
    
            if y_true[x, y] == 1: # if positive and unsure
                if y_true_sum < 9: # if edge and there is a candidate
                    diffs = y_pred_max - y_pred[x, y]
                    percentage = np.min([1/(1 + np.exp(-(diffs * 10 - 5))), 0.5])
                    y_out[x, y] += percentage
                    change_map[x, y] = percentage
            if y_true[x, y] == 0:# and y_pred[x, y] > 0.75: # if negative and unsure
                if y_true_sum > 0: #and y_pred_min < 0.25: # if edge and there is a candidate
                    diffs = y_pred[x, y] - y_pred_min
                    percentage = np.min([1/(1 + np.exp(-(diffs * 10 - 5))), 0.5])
                    y_out[x, y] -= percentage #= np.min(y_pred[min_x:max_x, min_y:max_y])
                    change_map[x, y] = -percentage
    return y_out, change_map



In [ ]:
start = 0

In [ ]:
from sklearn.metrics import f1_score
print(start/len(test_ids))
#test_ids = [x for x in range(test_x.shape[0])]
#test_ids = sorted(test_ids)
#test_ids = np.argwhere(abs(np.array(diffs)) > 40)
matrix_ids = [test_ids[start], test_ids[start + 1], test_ids[start + 2], test_ids[start + 3],
              test_ids[start + 4], test_ids[start + 5], test_ids[start + 6], test_ids[start + 7]]


preds = []
trues = []
for i in matrix_ids:
    idx = i
    #print(i)
    x_input = test_x[idx].reshape(1, 12, IMAGE_SIZE, IMAGE_SIZE, n_bands)
    median_input = calc_median_input(x_input)
    y = sess.run([fm], feed_dict={inp: x_input,
                                  inp_median: median_input,
                                  length: np.full((1, 1), 12),
                                  is_training: False,
                                  clipping_params['rmax']: rmax_epoch,
                                  clipping_params['rmin']: rmin_epoch,
                                  clipping_params['dmax']: dmax_epoch,
                                  })
    y = np.array(y).reshape(14, 14)
    #y, mapshape = aggregate_maxes(test_y[idx], y)
    preds.append(y)
    y2 = np.copy(y)
    #print(idx, list(test_data.iloc[i, 0])[0],
    #      list(test_data.iloc[i, 1])[0],
    #      list(test_data.iloc[i, 2])[0], diffs_new[i[0]])
    print(idx, test_data.iloc[i, 0],
          test_data.iloc[i, 1],
          test_data.iloc[i, 2])#, diffs[i[0]])
    true = test_y[idx].reshape(14, 14)
    trues.append(true)


to_plot = trues[0:4] + preds[0:4] + trues[4:] + preds[4:]

multiplot(to_plot, nrows = 4, ncols = 4)


start = start + 8 

# 123, 334, 680, 875, 917, 950

In [ ]:
train_ids = [x for x in range(len(train_y))]
diffs = []
for idx in tnrange(len(train_ids)):
    x_input = train_x[idx].reshape(1, 12, IMAGE_SIZE, IMAGE_SIZE, n_bands)
    median_input = calc_median_input(x_input)
    y = sess.run([fm], feed_dict={inp: x_input,
                                  inp_median: median_input,
                                  length: train_l[idx].reshape(1, 1),
                                  is_training: False,
                                  clipping_params['rmax']: rmax_epoch,
                                  clipping_params['rmin']: rmin_epoch,
                                  clipping_params['dmax']: dmax_epoch,
                                  })
    y = np.array(y).reshape(14, 14)
    y[np.where(y > 0.45)] = 1.0
    y[np.where(y < 0.45)] = 0.
    diff = np.sum(y) - np.sum(train_y[idx])
    diffs.append(diff)

In [ ]:
train_ids = [x for x in range(train_x.shape[0])]
train_ids = sorted(train_ids)
start = len(train_ids) - 16




In [ ]:
##matrix_ids = [70, 139, 456, 630, 718, 800, 835, 1101]

#train_ids = np.argwhere(np.logical_and(abs(np.array(diffs)) >= 50,
#                                       abs(np.array(diffs)) < 60))

matrix_ids = [train_ids[start], train_ids[start + 1], train_ids[start + 2],
             train_ids[start + 3], train_ids[start + 4],
             train_ids[start + 5], train_ids[start + 6], train_ids[start + 7]]
#matrix_ids = to_update[-8:]
preds = []
trues = []
#print(start//4)
print(matrix_ids)
for i in matrix_ids:
    idx = i
    x_input = train_x[idx].reshape(1, 12, IMAGE_SIZE, IMAGE_SIZE, n_bands)
    median_input = calc_median_input(x_input)
    y = sess.run([fm], feed_dict={inp: x_input,
                                  inp_median: median_input,
                                  length: np.full((1, 1), 12),
                                  is_training: False,
                                  clipping_params['rmax']: rmax_epoch,
                                  clipping_params['rmin']: rmin_epoch,
                                  clipping_params['dmax']: dmax_epoch,
                                    })
    y = np.array(y).reshape(14, 14)
    
    #y, _ = aggregate_maxes(train_y[idx], y)
    
    preds.append(y)
    true = train_y[idx].reshape(14, 14)
    
    #print(idx, (list(data.iloc[idx, 1])[0], list(data.iloc[idx, 2])[0]), diffs[i[0]])
    #print(idx, test_data.iloc[i, 0], test_data.iloc[i, 1],
    #      test_data.iloc[i, 2], diffs[i[0]])
    trues.append(true)
    
start += 8

to_plot = trues[0:4] + preds[0:4] + trues[4:] + preds[4:]
multiplot(to_plot, nrows = 4, ncols = 4)